In [ ]:
import pandas as pd
import numpy as np
symbol = 'Security 1'

In [ ]:
price_data = pd.DataFrame(np.cumsum(np.random.randn(150, 2).dot([[0.5, 0.4], [0.4, 1.0]]), axis=0) + 100,
                          columns=['Security 1', 'Security 2'], index=pd.date_range(start='01-01-2007', periods=150))

dates_actual = price_data.index.values
prices = price_data[symbol].values

In [ ]:
from bqplot import *
from bqplot.interacts import *
from traitlets import link

from IPython.display import display
from ipywidgets import Latex, ToggleButtons, VBox

In [ ]:
## call back for selectors
def interval_change_callback(name, value):
    db.value = str(value)

# Line Chart Selectors

## Fast Interval Selector

In [ ]:
dt_x = DateScale()
# lin_x = LinearScale(min=-50, max=len(dates) + 50)
lin_y = LinearScale()

x_ax = Axis(label="Index", scale = dt_x)
x_ay = Axis(label=(symbol + " Price"), scale = lin_y, orientation="vertical", tick_format="0.2f")
lc = Lines(x=dates_actual, y=prices, scales={'x': dt_x, 'y': lin_y}, colors=['orange'])
lc_2 = Lines(x=dates_actual[50:], y=prices[50:] + 2, scales={'x': dt_x, 'y': lin_y}, colors=['yellow'])

intsel = FastIntervalSelector(scale=dt_x, marks=[lc, lc_2])
intsel.on_trait_change(interval_change_callback, name='selected')

db = Latex()
db.value = str(intsel.selected)
display(db)

fig = Figure(marks=[lc, lc_2], axes=[x_ax, x_ay], title="Ford Historic Time Series", interaction=intsel)
display(fig)

## Index Selector

In [ ]:
index_sel = IndexSelector(scale=dt_x, marks=[lc, lc_2])
index_sel.on_trait_change(interval_change_callback, name='selected')
fig.interaction = index_sel

In [ ]:
## call back for selectors
def date_interval_change_callback(name, value):
    db_date.value = str(value)

In [ ]:
from datetime import datetime as py_dtime

dt_x = DateScale(min=np.datetime64(py_dtime(2006, 6, 1)))
lc2_y = LinearScale()

lc2 = Lines(x=dates_actual, y=prices,
            scales={'x': dt_x, 'y': lc2_y})

x_ax1 = Axis(label="Date", scale = dt_x)
x_ay2 = Axis(label=(symbol + " Price"), scale = lc2_y, orientation="vertical", tick_format="0.2f")

intsel_date = FastIntervalSelector(scale=dt_x, marks=[lc2])
db_date = Latex()
db_date.value = str(intsel_date.selected)
display(db_date)
lc2.on_trait_change(date_interval_change_callback, name='selected')

fig = Figure(marks=[lc2], axes=[x_ax1, x_ay2], interaction=intsel_date)
display(fig)

## Brush Selector

In [ ]:
intsel_date = BrushIntervalSelector(scale=dt_x, marks=[lc2], color='blue')
lc2.on_trait_change(date_interval_change_callback, name='selected')
fig.interaction = intsel_date

# Scatter Chart Selectors

In [ ]:
symbol2 = 'Security 2'
date_fmt = '%m-%d-%Y'

sec2_data = price_data[symbol2].values
dates = price_data.index.values

In [ ]:
## call back for the selector
def brush_callback(name, value):
    if(not br_sel.brushing):
        db2.value = str(br_sel.selected)

## Brush Selector

In [ ]:
sc_x = LinearScale()
sc_y = LinearScale()

scatt = Scatter(x=prices, y=sec2_data,
                scales={'x': sc_x, 'y': sc_y})

br_sel = BrushSelector(x_scale=sc_x, y_scale=sc_y, marks=[scatt], color='red')
br_sel.on_trait_change(brush_callback, name='brushing')

db2 = Latex()
db2.value = str(br_sel.selected)
display(db2)

sc_xax = Axis(label=(symbol), scale=sc_x, tick_format="0.0f")
sc_yax = Axis(label=(symbol2), scale=sc_y, orientation="vertical", tick_format="0.0f")
fig = Figure(marks =[scatt], axes=[sc_xax, sc_yax], interaction=br_sel)
display(fig)

## Brush Selector with Date Values

In [ ]:
sc_date_x = DateScale(date_format=date_fmt)
sc_y = LinearScale()

scatt = Scatter(x=dates_actual, y=sec2_data,
                scales={'x': sc_date_x, 'y': sc_y})
br_sel = BrushSelector(x_scale=sc_date_x, y_scale=sc_y, marks=[scatt])
br_sel.on_trait_change(brush_callback, name='brushing')

db2 = Latex()
db2.value = str(br_sel.selected)
display(db2)

sc_xax = Axis(label=(symbol), scale=sc_date_x)
sc_yax = Axis(label=(symbol2), scale=sc_y, orientation="vertical", tick_format="0.0f")
fig = Figure(marks =[scatt], axes=[sc_xax, sc_yax], interaction=br_sel)
display(fig)

# Histogram Selectors

In [ ]:
## call back for selectors
def interval_change_callback(name, value):
    db3.value = str(value)
    
## call back for the selector
def brush_callback(name, value):
    if(not br_intsel.brushing):
        db3.value = str(br_intsel.selected)

In [ ]:
returns = np.log(prices[1:]) - np.log(prices[:-1])
hist_x = LinearScale()
hist_y = LinearScale()
hist = Hist(sample=returns, scales={'sample': hist_x, 'count': hist_y})

br_intsel = BrushIntervalSelector(scale=hist_x, marks=[hist])
br_intsel.on_trait_change(brush_callback, name='selected')
br_intsel.on_trait_change(brush_callback, name='brushing')

db3 = Latex()
db3.value = str(br_intsel.selected)
display(db3)

h_xax = Axis(scale=hist_x, label='Returns', grids='off', set_ticks=True, tick_format='0.2%')
h_yax = Axis(scale=hist_y, label='Freq', orientation='vertical', grids='off')

fig2 = Figure(marks=[hist], axes=[h_xax, h_yax], interaction=br_intsel)
display(fig2)

## Multi Selector

In [ ]:
def multi_sel_callback(name, value):
    if(not multi_sel.brushing):
        db3.value = str(multi_sel.selected)

In [ ]:
line_x = LinearScale()
line_y = LinearScale()
line = Lines(x=np.arange(100), y=np.random.randn(100), scales={'x': line_x, 'y': line_y})

multi_sel = MultiSelector(scale=line_x, marks=[line])
multi_sel.on_trait_change(multi_sel_callback, name='selected')
multi_sel.on_trait_change(multi_sel_callback, name='brushing')

db3 = Latex()
db3.value = str(multi_sel.selected)
display(db3)

h_xax = Axis(scale=line_x, label='Returns', grids='off', set_ticks=True)
h_yax = Axis(scale=hist_y, label='Freq', orientation='vertical', grids='off')

fig2 = Figure(marks=[line], axes=[h_xax, h_yax], interaction=multi_sel)
display(fig2)

In [ ]:
# changing the names of the intervals.
multi_sel.names = ['int1', 'int2', 'int3']

## Multi Selector with Date X

In [ ]:
dt_x = DateScale(min=np.datetime64(py_dtime(2007, 1, 1)))
line_y = LinearScale()
line = Lines(x=dates_actual, y=sec2_data, scales={'x': dt_x, 'y': line_y}, colors=['red'])

multi_sel = MultiSelector(scale=dt_x)
multi_sel.on_trait_change(multi_sel_callback, name='selected')
multi_sel.on_trait_change(multi_sel_callback, name='brushing')

db3 = Latex()
db3.value = str(multi_sel.selected)
display(db3)

h_xax = Axis(scale=dt_x, label='Returns', grids='off')
h_yax = Axis(scale=line_y, label='Freq', orientation='vertical', grids='off')

fig2 = Figure(marks=[line], axes=[h_xax, h_yax], interaction=multi_sel)
display(fig2)

## Lasso Selector

In [ ]:
lasso_sel = LassoSelector()

In [ ]:
xs, ys = LinearScale(), LinearScale()
data = np.arange(20)
line = Lines(x=data, y=data, scales={'x': xs, 'y': ys})
scatter = Scatter(x=data, y=data, scales={'x': xs, 'y': ys}, colors=['skyblue'])
bar = Bars(x=data, y=data/2., scales={'x': xs, 'y': ys})
xax, yax = Axis(scale=xs, label='X'), Axis(scale=ys, label='Y', orientation='vertical')
fig = Figure(marks=[scatter, line, bar], axes=[xax, yax], interaction=lasso_sel)
lasso_sel.marks = [scatter, line]
display(fig)

In [ ]:
scatter.selected, line.selected

## Pan Zoom

In [ ]:
xs = DateScale(min=np.datetime64(py_dtime(2007, 1, 1)))
ys = LinearScale()
line = Lines(x=dates_actual, y=sec2_data, scales={'x': xs, 'y': ys}, colors=['red'])

panzoom = PanZoom(scales={'x': [xs], 'y': [ys]})
xax = Axis(scale=xs, label='Date', grids='off')
yax = Axis(scale=ys, label='Price', orientation='vertical', grids='off')

fig = Figure(marks=[line], axes=[xax, yax], interaction=panzoom)
display(fig)

## Hand Draw

In [ ]:
xs = DateScale(min=np.datetime64(py_dtime(2007, 1, 1)))
ys = LinearScale()
line = Lines(x=dates_actual, y=sec2_data, scales={'x': xs, 'y': ys}, colors=['red'])

handdraw = HandDraw(lines=line)
xax = Axis(scale=xs, label='Date', grids='off')
yax = Axis(scale=ys, label='Price', orientation='vertical', grids='off')

fig = Figure(marks=[line], axes=[xax, yax], interaction=handdraw)
display(fig)

# Unified Figure with All Interactions

In [ ]:
dt_x = DateScale(date_format=date_fmt, min=py_dtime(2007, 1, 1))
lc1_x = LinearScale()
lc2_y = LinearScale()

lc2 = Lines(x=np.linspace(0.0, 10.0, len(prices)), y=prices * 0.25,
            scales={'x': lc1_x, 'y': lc2_y}, 
            display_legend=True,
            labels=['Security 1'])

lc3 = Lines(x=dates_actual, y=sec2_data,
            scales={'x': dt_x, 'y': lc2_y},
            colors=['red'], 
            display_legend=True, 
            labels=['Security 2'])

lc4 = Lines(x=np.linspace(0.0, 10.0, len(prices)), y=sec2_data * 0.75,
            scales={'x': LinearScale(min=5, max=10), 'y': lc2_y},
            colors=['green'], display_legend=True, 
            labels=['Security 2 squared'])

x_ax1 = Axis(label='Date', scale=dt_x)
x_ax2 = Axis(label='Time', scale=lc1_x, side='top')
x_ay2 = Axis(label=(symbol + ' Price'), scale=lc2_y, orientation='vertical', tick_format='0.2f')


fig = Figure(marks=[lc2, lc3, lc4], axes=[x_ax1, x_ax2, x_ay2])

In [ ]:
## declaring the interactions
multi_sel = MultiSelector(scale=dt_x, marks=[lc2, lc3])
br_intsel = BrushIntervalSelector(scale=lc1_x, marks=[lc2, lc3])
index_sel = IndexSelector(scale=dt_x, marks=[lc2, lc3])
int_sel = FastIntervalSelector(scale=dt_x, marks=[lc3, lc2])

hd = HandDraw(lines=lc2)
hd2 = HandDraw(lines=lc3)
pz = PanZoom(scales={'x': [dt_x], 'y': [lc2_y]})

deb = Latex()
deb.value = "hello"

In [ ]:
## Call back handler for the interactions
def test_callback(name, value):
    deb.value = str(value)
    
multi_sel.on_trait_change(test_callback, name='selected')
br_intsel.on_trait_change(test_callback, name='selected')
index_sel.on_trait_change(test_callback, name='selected')
int_sel.on_trait_change(test_callback, name='selected')

In [ ]:
from collections import OrderedDict
selection_interacts = ToggleButtons(options=OrderedDict([('HandDraw1', hd), ('HandDraw2', hd2), ('PanZoom', pz), 
                                                       ('FastIntervalSelector', int_sel), ('IndexSelector', index_sel),
                                                       ('BrushIntervalSelector', br_intsel), ('MultiSelector', multi_sel),
                                                       ('None', None)]))
display(deb)
display(VBox([fig, selection_interacts], align_self="stretch"))

link((selection_interacts, 'value'), (fig, 'interaction'));

In [ ]:
# Set the scales of lc4 to the ones of lc2 and check if panzoom pans the two.
lc4.scales = lc2.scales